# Camada SILVER – Análise Exploratória de Dados

## Descrição Geral

Este notebook tem como propósito realizar o **carregamento e a análise exploratória** dos dados inseridos na **camada SILVER** da Data Layer.  
Nessa etapa, são utilizados os arquivos que já passaram por processos de **limpeza, padronização e enriquecimento** na camada anterior (RAW).
Os dados estão estruturados, validados e prontos para análises mais aprofundadas. O objetivo é compreender as relações entre entidades, identificar padrões e preparar insights que serão utilizados nas análises finais da camada Gold.

## Configuração Inicial

A primeira parte do notebook é dedicada à preparação do ambiente de trabalho, incluindo:

- **Importação das bibliotecas essenciais**, como `pandas`, `numpy`, `matplotlib` e `seaborn`, com breve explicação de seus papéis na análise.  
- **Definição dos caminhos** para os arquivos localizados na camada SILVER.  
- **Levantamento de metadados** iniciais, como tipos de dados, quantidade de registros, integridade referencial e qualidade dos dados após o pré-processamento.

Ao final desta etapa, espera-se ter uma visão clara sobre a **qualidade, estrutura e relacionamentos dos dados processados**, bem como identificar oportunidades para análises mais complexas na próxima camada.
